In [ ]:
import os
import re
import json
import random
import pandas as pd

In [59]:
parent_dir = "raw/processed_magpie"
filename = "magpieclean_1k.jsonl"
filepath = os.path.join(parent_dir, filename)

# Load data agnostic to parquet / json / jsonl
if filename.endswith('.parquet'):
    df = pd.read_parquet(filepath)
    data = df.to_dict('records')
    print(f"Loaded {len(data)} items from parquet file {filename}.")
elif filename.endswith('.json'):
    with open(filepath, 'r') as f:
        data = json.load(f)
    print(f"Loaded {len(data)} items from JSON file {filename}.")
elif filename.endswith('.jsonl'):
    data = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    print(f"Loaded {len(data)} items from JSONL file {filename}.")
else:
    raise ValueError(f"Unsupported file format: {filename}. Please use .parquet, .json, or .jsonl files.")

random.shuffle(data)
print(f"Shuffled {len(data)} items.")

Loaded 1000 items from JSONL file magpieclean_1k.jsonl.
Shuffled 1000 items.


In [65]:
# Randomly sample / print data
sample = random.choice(data)
# print(sample)

for role, content in sample['chat']:
    print(f"Role: {role}\n{'-'*20}\n{content}\n\n")

if 'info' in sample:
    print(f"Info:\n{'-'*20}\n{sample['info']}")

Role: system
--------------------
llama4_default_sysprompt.txt


Role: user
--------------------
### Introduction to Relational Databases


Role: assistant
--------------------
**Introduction to Relational Databases**

A relational database is a type of database that stores and provides access to data that is related to each other. It is based on the relational model, which was introduced by Edgar F. Codd in 1969. Relational databases are widely used in many applications, including web applications, mobile apps, and enterprise software.

**Key Features of Relational Databases**
--------------------------------------

1. **Tables**: Relational databases store data in tables, which are also known as relations. Each table has rows and columns, where each row represents a single record and each column represents a field or attribute of that record.
2. **Columns**: Each column in a table has a specific data type, such as integer, string, or date.
3. **Primary Key**: Each table has a primary

## Code to clean up / concatenate existing data

In [35]:
# Example usage:
parent_dir = "explainer_data"
filenames = [
    "explainer_clean_100_1558_15.parquet",
    "explainer_clean_1250.parquet",
    "explanations_0_1000_0104_16.parquet",
    "explanations_1_1000_0330_16.parquet",
    "explanations_2_1000_0557_16.parquet",
    "explanations_3_1000_0826_16.parquet"
]
filepaths = [os.path.join(parent_dir, fn) for fn in filenames]
output_file = "combined_chessexplainer.jsonl"
system_prompt_file = "chess_task_sysprompt.txt"  # Or "llama4_default_sysprompt.txt"


HEADER_PATTERN = re.compile(
    r"<\|start_header_id\|>(\w+)<\|end_header_id\|>\n?(.*?)(?=(<\|start_header_id\|>|\Z|<\|eot_id\|>))",
    re.DOTALL
)

def extract_dialogue(sample):
    text = sample['prompt']
    result = []
    for match in HEADER_PATTERN.finditer(text):
        role, content = match.group(1), match.group(2).strip()
        # Remove Llama tags and both eot_id types
        content = re.sub(r"<\|.*?\|>|<eot_id>", "", content).strip()
        if role == "system":
            content = "chess_task_sysprompt.txt"
        if role == "user":
            prefix = "Here is a board in a game you're currently playing. I want you to think through some possible moves you could make and how those moves will likely play out. You may find it helpful to roll-out each line assuming your opponent plays near-optimally. You may also find it helpful to consider the value of the final board state after each roll-out.\n\nAfter you think through your various moves, please end by telling me your chosen move (in UCI notation) within answer tags.\n\n"
            content = prefix + content
        if content:
            result.append((role, content))
    completion = sample.get('completion', '').strip()
    completion = re.sub(r'(<\|eot_id\|>|<eot_id>)\s*$', '', completion).strip()
    if completion:
        result.append(('assistant', completion))
    return result

def convert_and_save(parquet_paths, output_path):
    all_dialogues = []
    for path in parquet_paths:
        df = pd.read_parquet(path)
        all_dialogues.extend(
            {"chat": extract_dialogue(row)} for row in df.to_dict('records')
        )
    # Save as JSONL
    with open(output_path, 'w', encoding='utf-8') as f:
        for d in all_dialogues:
            json.dump(d, f, ensure_ascii=False)
            f.write('\n')
    print(f"Saved {len(all_dialogues)} dialogues to {output_path}")


output_path = os.path.join(parent_dir, output_file)
convert_and_save(filepaths, output_path)

Saved 5350 dialogues to explainer_data\combined_chessexplainer.jsonl
